In [2]:
import spacy
# en_nlp = spacy.load('en')
# from spacy.en import English
# parser = English()

import datetime
import pandas as pd
import numpy as np
import cPickle as pickle
from collections import defaultdict
from gensim import corpora, models, matutils



In [3]:
with open('master_total_df.p','rb') as f:
    master_total_df = pickle.load(f)

In [4]:
master_total_df.head(2)

,index,company,jobdesc,city,state,title,sourcesite,views,base_title,parsed_title,parsed_title_i,expanded_title,prefix_title
0,0,Advocate Health Care,"\nAs part of Advocate Health Care, Advocate Ch...",Oak Lawn,IL,Clinical Practice Specialist - 4 Hope,ind,0,specialist,"[clinical, practice, specialist, , , 4, hope]",[2],practice specialist,practice
1,1,University of Washington Medical Center,\nThe University of Washington (UW) is proud t...,Seattle,WA,WEB DEVELOPER,ind,0,developer,"[web, developer]",[1],web developer,web


# Sample small test set of Job Postings from the others

In [5]:
master_sample = master_total_df.head(10)
all_text = master_sample['jobdesc']
all_text.head(2)

0    \nAs part of Advocate Health Care, Advocate Ch...
1    \nThe University of Washington (UW) is proud t...
Name: jobdesc, dtype: object

### Cleaning up the data for ascii characters to use spacy splitter

In [6]:
#only ascii
import string
alphas = string.ascii_lowercase+' .'
print alphas

clean_words_text = [''.join([x.lower() for x in text if (ord(x.lower())<128) & (x.lower() in alphas)])  for text in all_text]
clean_words_text = [ x.replace('.', '. ').replace('.','') for x in clean_words_text]

abcdefghijklmnopqrstuvwxyz .


In [7]:
import spacy
from spacy.en import English
parser = English()
vectorizer = CountVectorizer(stop_words='english')
#vectorizer = TFIDFVectorizer(stop_words='english')
stopw = [x for x in vectorizer.get_stop_words()]

NameError: name 'CountVectorizer' is not defined

In [ ]:
#tokenizing
token_text = []
start_time = datetime.datetime.now()
for text in clean_words_text: 
    test = text

    parsedData = parser(unicode(test))
    clean_tokens = [x.string.strip() for x in parsedData if len(x.string.strip())>0]
    token_text.append(clean_tokens)
print (datetime.datetime.now() - start_time)



In [ ]:
stopw

In [166]:
# This is the fastest way to swap a dictionary key / value order.  This is the format gensim LDA expects it's vocabulary.
# remove words that appear only once
frequency = defaultdict(int)


for text in token_text:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if (frequency[token] > 1) & (token not in stopw)]
          for text in token_text]

# texts = [[token for token in text if frequency[token] > 1]
#           for text in documents]

# Create gensim dictionary object
dictionary = corpora.Dictionary(texts)

# Create corpus matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [168]:
texts[:20]

[[u'advocate',
  u'health',
  u'care',
  u'advocate',
  u'childrens',
  u'hospital',
  u'largest',
  u'network',
  u'provider',
  u'pediatric',
  u'services',
  u'illinois',
  u'nation',
  u'main',
  u'chicago',
  u'metropolitan',
  u'region',
  u'campuses',
  u'located',
  u'oak',
  u'southwest',
  u'park',
  u'northwest',
  u'additional',
  u'services',
  u'accessible',
  u'advocate',
  u'hospitals',
  u'state',
  u'advocate',
  u'childrens',
  u'hospital',
  u'serves',
  u'major',
  u'referral',
  u'center',
  u'children',
  u'special',
  u'approach',
  u'advocate',
  u'childrens',
  u'hospital',
  u'combines',
  u'countrys',
  u'respected',
  u'medical',
  u'talent',
  u'exceptional',
  u'compassionate',
  u'care',
  u'fact',
  u'childrens',
  u'hospital',
  u'recognized',
  u'u',
  u's',
  u'news',
  u'world',
  u'report',
  u'nations',
  u'leaders',
  u'pediatric',
  u'numerous',
  u'physicians',
  u'staff',
  u'recognized',
  u'doctors',
  u'regionally',
  u'nationally',
  u'fie

# Model with 3 topics

In [176]:
start_time = datetime.datetime.now()
lda = models.LdaModel(
    corpus,
    num_topics  =  3,
    passes      =  15,
    id2word     =  dictionary
)
print datetime.datetime.now()-start_time

0:01:32.818476


In [177]:
lda.print_topics(num_topics=3, num_words=8)

[(0,
  u'0.028*data + 0.015*experience + 0.009*business + 0.007*team + 0.007*work + 0.006*years + 0.005*skills + 0.005*design'),
 (1,
  u'0.011*marketing + 0.009*team + 0.008*experience + 0.007*work + 0.007*business + 0.006*analytics + 0.005*new + 0.005*data'),
 (2,
  u'0.017*business + 0.010*experience + 0.009*management + 0.007*work + 0.007*data + 0.007*project + 0.007*requirements + 0.006*skills')]

In [ ]:
lda.get_document_topics(corpus[0])

# Model with 5 topics

In [178]:
start_time = datetime.datetime.now()
lda = models.LdaModel(
    corpus,
    num_topics  =  5,
    passes      =  20,
    id2word     =  dictionary
)
print datetime.datetime.now()-start_time

0:02:03.666048


In [179]:
lda.print_topics(num_topics=5, num_words=8)

[(0,
  u'0.017*business + 0.009*experience + 0.007*team + 0.007*management + 0.007*work + 0.006*data + 0.006*support + 0.006*financial'),
 (1,
  u'0.031*data + 0.015*experience + 0.009*business + 0.009*team + 0.008*work + 0.006*software + 0.006*development + 0.006*skills'),
 (2,
  u'0.009*data + 0.009*experience + 0.009*business + 0.008*health + 0.007*work + 0.007*systems + 0.006*management + 0.006*analysis'),
 (3,
  u'0.018*marketing + 0.010*data + 0.009*analytics + 0.008*experience + 0.008*team + 0.008*digital + 0.007*work + 0.007*business'),
 (4,
  u'0.017*business + 0.013*experience + 0.011*requirements + 0.011*management + 0.010*project + 0.007*support + 0.007*team + 0.006*development')]

# Model with 10 topics

In [180]:
start_time = datetime.datetime.now()
lda = models.LdaModel(
    corpus,
    num_topics  =  10,
    passes      =  50,
    id2word     =  dictionary
)
print datetime.datetime.now()-start_time

0:05:09.317015


In [181]:
lda.print_topics(num_topics=10, num_words=8)

[(0,
  u'0.033*data + 0.016*experience + 0.010*team + 0.009*work + 0.008*business + 0.007*analytics + 0.006*software + 0.006*development'),
 (1,
  u'0.024*supply + 0.021*chain + 0.011*sourcing + 0.010*analytics + 0.007*manufacturing + 0.006*business + 0.006*planning + 0.005*team'),
 (2,
  u'0.015*business + 0.010*experience + 0.009*requirements + 0.009*management + 0.008*project + 0.008*data + 0.008*support + 0.007*work'),
 (3,
  u'0.008*experience + 0.007*work + 0.006*solutions + 0.006*management + 0.006*program + 0.006*business + 0.005*client + 0.005*services'),
 (4,
  u'0.016*experience + 0.011*security + 0.011*systems + 0.008*analysis + 0.008*management + 0.008*information + 0.007*requirements + 0.007*related'),
 (5,
  u'0.019*business + 0.010*marketing + 0.010*data + 0.009*experience + 0.009*management + 0.009*team + 0.008*skills + 0.007*analysis'),
 (6,
  u'0.018*health + 0.015*care + 0.009*experience + 0.008*clinical + 0.008*data + 0.007*healthcare + 0.007*research + 0.007*hospi

# with ngram 2

In [222]:
# will have to be a bit slower / can't use the normal approach

# This is the fastest way to swap a dictionary key / value order.  This is the format gensim LDA expects it's vocabulary.
vectorizer = CountVectorizer(stop_words='english', ngram_range=(2,2))
#vectorizer = TFIDFVectorizer(stop_words='english')
X = vectorizer.fit_transform(all_text)

In [224]:
docs = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
counts = docs.sum().transpose().sort_values(ascending=False)
exclude = counts[counts==1].index
print len(exclude)
start_time = datetime.datetime.now()
print 'starting now'

docs.drop(exclude,axis=1,inplace=True)
print 'finished', datetime.datetime.now() - start_time

85003
starting now
finished 0:00:00.219140


In [228]:
docs = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
# This is the fastest way to swap a dictionary key / value order.  This is the format gensim LDA expects it's vocabulary.
vocab = {v: k for k, v in vectorizer.vocabulary_.iteritems()}
print vocab
#twogram_corpus = matutils.Sparse2Corpus(X, documents_columns=False)


{0: u'00 00', 1: u'00 10', 2: u'00 flsa', 3: u'00 hourjob', 4: u'00 hr', 5: u'00 pm', 6: u'00 regular', 7: u'00 shift', 8: u'00 year', 9: u'00 yearjob', 10: u'000 00', 11: u'000 110', 12: u'000 admissions', 13: u'000 associates', 14: u'000 caregivers', 15: u'000 caterers', 16: u'000 clients', 17: u'000 customers', 18: u'000 dimensions', 19: u'000 direct', 20: u'000 emergency', 21: u'000 employees', 22: u'000 engaged', 23: u'000 extraordinary', 24: u'000 factset', 25: u'000 feet', 26: u'000 financial', 27: u'000 flexible', 28: u'000 games', 29: u'000 hours', 30: u'000 key', 31: u'000 leaders', 32: u'000 lives', 33: u'000 members', 34: u'000 mobile', 35: u'000 month', 36: u'000 organizations', 37: u'000 owner', 38: u'000 people', 39: u'000 physicians', 40: u'000 professionals', 41: u'000 receipt', 42: u'000 restaurant', 43: u'000 restaurants', 44: u'000 retail', 45: u'000 role', 46: u'000 route', 47: u'000 service', 48: u'000 shopping', 49: u'000 skilled', 50: u'000 students', 51: u'000 

In [ ]:

lda = models.LdaModel(
    twogram_corpus,
    # or use the corpus object created with the dictionary in the previous frame!
    # corpus, 
    num_topics  =  3,
    passes      =  20,
    id2word     =  vocab
    # or use the gensim dictionary object!
    # id2word     =  dictionary
)